In [1]:
print("Hello World")

Hello World


In [3]:
!pip install torch

  Using cached torch-1.13.1-cp38-cp38-manylinux1_x86_64.whl (887.4 MB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-nightly 2.12.0.dev20221110 requires astunparse>=1.6.0, which is not installed.
tf-nightly 2.12.0.dev20221110 requires gast<=0.4.0,>=0.2.1, which is not installed.
tf-nightly 2.12.0.dev20221110 requires google-pasta>=0.1.1, which is not installed.
tf-nightly 2.12.0.dev20221110 requires grpcio<2.0,>=1.24.3, which is not installed.
tf-nig

In [6]:
!pip install auto-sklearn

  Using cached auto-sklearn-0.15.0.tar.gz (6.5 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached distro-1.8.0-py3-none-any.whl (20 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 9.1 MB/s eta 0:00:000m eta 0:00:010:01:01
  Using cached pynisher-0.6.4.tar.gz (11 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 8.8 MB/s eta 0:00:000m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 951.2/951.2 kB 7.4 MB/s eta 0:00:00m eta 0:00:010:01
  Using cached smac-1.2.tar.gz (260 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.8 MB/s eta 0:00:000m eta 0:00:010:00:01
  Using cached PyYAML-6.0-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (701 kB)
  Using cached liac-arff-2.5.0.tar.gz (13 kB)
  Preparing metadata (

Hello World


In [7]:
import numpy as np
import pickle
import torch
import time
import autosklearn

from sklearn.metrics import r2_score, mean_absolute_error, explained_variance_score, mean_squared_error
from sklearn.model_selection import train_test_split
import autosklearn.regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
plt.matplotlib.style.use("https://gist.githubusercontent.com/JonnyCBB/c464d302fefce4722fe6cf5f461114ea/raw/64a78942d3f7b4b5054902f2cee84213eaff872f/matplotlibrc")
cool_colors = ['#00BEFF', '#D4CA3A', '#FF6DAE', '#67E1B5', '#EBACFA', '#9E9E9E', '#F1988E', '#5DB15A', '#E28544', '#52B8AA']
cool_colors = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7']

plt.rcParams.update({'font.size': 14})

load data from `prepare_Xy.ipynb`

In [10]:
X = pickle.load(open('hydrogen_input_output.pkl', 'rb'))['x']
print("shape of X:", np.shape(X))

y = pickle.load(open('hydrogen_input_output.pkl', 'rb'))['y']
print("shape of y:", np.shape(y))

nb_COFs = pickle.load(open('hydrogen_input_output.pkl', 'rb'))['nb_COFs']
print("# COFs:", nb_COFs)

nb_iterations = pickle.load(open('hydrogen_input_output.pkl', 'rb'))['nb_iterations']
print("# iterations:", nb_iterations)

nb_runs = pickle.load(open('hydrogen_input_output.pkl', 'rb'))['nb_runs']
print("# runs:", nb_runs)

# feature names
features = pickle.load(open('hydrogen_input_output.pkl', 'rb'))['features']

shape of X: (98694, 7)
shape of y: (98694,)
# COFs: 98694
# iterations: 250
# runs: 100


set this flag to determined whether the random forest uses a diverse training set (`True`) or a randomly selected training set (`False`).

In [11]:
diversify_training = True

In [12]:
"""
select a diverse training set (the rest is testing data)
"""
def diverse_train_test_split(X, train_size):
    ids_train = [np.random.randint(0, nb_COFs)] # initialize with one random point; pick others in a max diverse fashion
    # select remaining training points
    for j in range(train_size - 1):
        # for each point in data set, compute its min distance to training set
        distances_to_train_set = np.linalg.norm(X - X[ids_train, None, :], axis=2)
        assert np.shape(distances_to_train_set) == (len(ids_train), nb_COFs)
        min_distances_to_a_training_pt = np.min(distances_to_train_set, axis=0)
        assert np.size(min_distances_to_a_training_pt) == nb_COFs
        
        # acquire point with max min distance to train set (Furthest from train set)
        ids_train.append(np.argmax(min_distances_to_a_training_pt))
    assert np.size(np.unique(ids_train)) == train_size # gotta be unique
    
    # test data is wutever's left
    ids_test = [i for i in range(nb_COFs) if not i in ids_train]
    assert np.size(np.unique(ids_test)) == nb_COFs - train_size
    
    return np.array(ids_train), np.array(ids_test)

a random forest run.

In [13]:
def rf_run(nb_training_data, nb_acquire):
    if diversify_training:
        print("\tdiverse RF run")
    else:
        print("\tRF run")
    print("\teval budget", nb_training_data + nb_acquire, "=", nb_training_data, "training data and", nb_acquire, "acquired.")
    
    # test/train split
    if diversify_training:
        ids_train, ids_test = diverse_train_test_split(X, nb_training_data)
    else:
        ids_train, ids_test = train_test_split(np.arange(nb_COFs), train_size=nb_training_data)
    
    X_train = X[ids_train, :]
    y_train = y[ids_train]
    
    X_test  = X[ids_test, :]
    
    # train random forest on training data
    rf = RandomForestRegressor()
    rf.fit(X_train, y_train)

    # hv random forest make predictions on test data
    y_pred = rf.predict(X_test)

    # rank the test predictions
    ids_test_ranked = np.flip(np.argsort(y_pred))
    
    # acquire the COFs in the test set with highest predicted property
    ids_acquire = ids_test[ids_test_ranked[:nb_acquire]]

    # return the acquired COFs but also the trained COFs which count.
    ids_acquire_incld_training = np.concatenate((ids_acquire, ids_train))
    
    assert np.size(np.unique(ids_acquire_incld_training)) == nb_training_data + nb_acquire
    
    print("\tmax y acquired = ", np.max(y[ids_acquire_incld_training]))
    return ids_acquire_incld_training

In [14]:
rf_res = dict()
rf_res['nb_evals_budgets'] = [20 * i for i in range(1, int(nb_iterations/20 + 1))] + [250]
print("eval budgets: ", rf_res['nb_evals_budgets'])
rf_res['ids_acquired']     = [[] for b in rf_res['nb_evals_budgets']]
for b in range(len(rf_res['nb_evals_budgets'])):
    nb_evals_budget = rf_res['nb_evals_budgets'][b]
    print("budget for evals:", nb_evals_budget)
    # decide how to spend the evals budget here. say 50/50
    nb_training_data = nb_evals_budget // 2
    nb_acquire = nb_evals_budget // 2
    assert nb_training_data + nb_acquire == nb_evals_budget
    
    for r in range(nb_runs):
        print("\trun", r)
        ids_acquired = rf_run(nb_training_data, nb_acquire)
        rf_res['ids_acquired'][b].append(ids_acquired)

eval budgets:  [20, 40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240, 250]
budget for evals: 20
	run 0
	diverse RF run
	eval budget 20 = 10 training data and 10 acquired.
	max y acquired =  45.71
	run 1
	diverse RF run
	eval budget 20 = 10 training data and 10 acquired.
	max y acquired =  45.71
	run 2
	diverse RF run
	eval budget 20 = 10 training data and 10 acquired.
	max y acquired =  45.71
	run 3
	diverse RF run
	eval budget 20 = 10 training data and 10 acquired.
	max y acquired =  45.71
	run 4
	diverse RF run
	eval budget 20 = 10 training data and 10 acquired.
	max y acquired =  45.71
	run 5
	diverse RF run
	eval budget 20 = 10 training data and 10 acquired.
	max y acquired =  45.71
	run 6
	diverse RF run
	eval budget 20 = 10 training data and 10 acquired.
	max y acquired =  45.71
	run 7
	diverse RF run
	eval budget 20 = 10 training data and 10 acquired.
	max y acquired =  45.71
	run 8
	diverse RF run
	eval budget 20 = 10 training data and 10 acquired.
	max y acquired =  45.71
	r

	max y acquired =  45.71
	run 81
	diverse RF run
	eval budget 20 = 10 training data and 10 acquired.
	max y acquired =  45.71
	run 82
	diverse RF run
	eval budget 20 = 10 training data and 10 acquired.
	max y acquired =  45.71
	run 83
	diverse RF run
	eval budget 20 = 10 training data and 10 acquired.
	max y acquired =  45.71
	run 84
	diverse RF run
	eval budget 20 = 10 training data and 10 acquired.
	max y acquired =  45.71
	run 85
	diverse RF run
	eval budget 20 = 10 training data and 10 acquired.
	max y acquired =  45.71
	run 86
	diverse RF run
	eval budget 20 = 10 training data and 10 acquired.
	max y acquired =  45.71
	run 87
	diverse RF run
	eval budget 20 = 10 training data and 10 acquired.
	max y acquired =  45.71
	run 88
	diverse RF run
	eval budget 20 = 10 training data and 10 acquired.
	max y acquired =  45.71
	run 89
	diverse RF run
	eval budget 20 = 10 training data and 10 acquired.
	max y acquired =  45.71
	run 90
	diverse RF run
	eval budget 20 = 10 training data and 10 

	max y acquired =  45.71
	run 63
	diverse RF run
	eval budget 40 = 20 training data and 20 acquired.
	max y acquired =  45.71
	run 64
	diverse RF run
	eval budget 40 = 20 training data and 20 acquired.
	max y acquired =  45.71
	run 65
	diverse RF run
	eval budget 40 = 20 training data and 20 acquired.
	max y acquired =  45.71
	run 66
	diverse RF run
	eval budget 40 = 20 training data and 20 acquired.
	max y acquired =  45.71
	run 67
	diverse RF run
	eval budget 40 = 20 training data and 20 acquired.
	max y acquired =  45.71
	run 68
	diverse RF run
	eval budget 40 = 20 training data and 20 acquired.
	max y acquired =  45.71
	run 69
	diverse RF run
	eval budget 40 = 20 training data and 20 acquired.
	max y acquired =  45.71
	run 70
	diverse RF run
	eval budget 40 = 20 training data and 20 acquired.
	max y acquired =  45.71
	run 71
	diverse RF run
	eval budget 40 = 20 training data and 20 acquired.
	max y acquired =  45.71
	run 72
	diverse RF run
	eval budget 40 = 20 training data and 20 

	max y acquired =  45.71
	run 45
	diverse RF run
	eval budget 60 = 30 training data and 30 acquired.
	max y acquired =  45.71
	run 46
	diverse RF run
	eval budget 60 = 30 training data and 30 acquired.
	max y acquired =  45.71
	run 47
	diverse RF run
	eval budget 60 = 30 training data and 30 acquired.
	max y acquired =  45.71
	run 48
	diverse RF run
	eval budget 60 = 30 training data and 30 acquired.
	max y acquired =  45.71
	run 49
	diverse RF run
	eval budget 60 = 30 training data and 30 acquired.
	max y acquired =  45.71
	run 50
	diverse RF run
	eval budget 60 = 30 training data and 30 acquired.
	max y acquired =  45.71
	run 51
	diverse RF run
	eval budget 60 = 30 training data and 30 acquired.
	max y acquired =  45.71
	run 52
	diverse RF run
	eval budget 60 = 30 training data and 30 acquired.
	max y acquired =  45.71
	run 53
	diverse RF run
	eval budget 60 = 30 training data and 30 acquired.
	max y acquired =  45.71
	run 54
	diverse RF run
	eval budget 60 = 30 training data and 30 

	max y acquired =  45.71
	run 27
	diverse RF run
	eval budget 80 = 40 training data and 40 acquired.
	max y acquired =  45.71
	run 28
	diverse RF run
	eval budget 80 = 40 training data and 40 acquired.
	max y acquired =  45.71
	run 29
	diverse RF run
	eval budget 80 = 40 training data and 40 acquired.
	max y acquired =  45.71
	run 30
	diverse RF run
	eval budget 80 = 40 training data and 40 acquired.
	max y acquired =  45.71
	run 31
	diverse RF run
	eval budget 80 = 40 training data and 40 acquired.
	max y acquired =  45.71
	run 32
	diverse RF run
	eval budget 80 = 40 training data and 40 acquired.
	max y acquired =  45.71
	run 33
	diverse RF run
	eval budget 80 = 40 training data and 40 acquired.
	max y acquired =  45.71
	run 34
	diverse RF run
	eval budget 80 = 40 training data and 40 acquired.
	max y acquired =  45.71
	run 35
	diverse RF run
	eval budget 80 = 40 training data and 40 acquired.
	max y acquired =  45.71
	run 36
	diverse RF run
	eval budget 80 = 40 training data and 40 

...

	max y acquired =  45.71
	run 73
	diverse RF run
	eval budget 200 = 100 training data and 100 acquired.
	max y acquired =  45.71
	run 74
	diverse RF run
	eval budget 200 = 100 training data and 100 acquired.
	max y acquired =  45.71
	run 75
	diverse RF run
	eval budget 200 = 100 training data and 100 acquired.
	max y acquired =  45.71
	run 76
	diverse RF run
	eval budget 200 = 100 training data and 100 acquired.
	max y acquired =  45.71
	run 77
	diverse RF run
	eval budget 200 = 100 training data and 100 acquired.
	max y acquired =  45.71
	run 78
	diverse RF run
	eval budget 200 = 100 training data and 100 acquired.
	max y acquired =  45.71
	run 79
	diverse RF run
	eval budget 200 = 100 training data and 100 acquired.
	max y acquired =  45.71
	run 80
	diverse RF run
	eval budget 200 = 100 training data and 100 acquired.
	max y acquired =  45.71
	run 81
	diverse RF run
	eval budget 200 = 100 training data and 100 acquired.
	max y acquired =  45.71
	run 82
	diverse RF run
	eval budget 20

	max y acquired =  45.71
	run 52
	diverse RF run
	eval budget 220 = 110 training data and 110 acquired.
	max y acquired =  45.71
	run 53
	diverse RF run
	eval budget 220 = 110 training data and 110 acquired.
	max y acquired =  45.71
	run 54
	diverse RF run
	eval budget 220 = 110 training data and 110 acquired.
	max y acquired =  45.71
	run 55
	diverse RF run
	eval budget 220 = 110 training data and 110 acquired.
	max y acquired =  45.71
	run 56
	diverse RF run
	eval budget 220 = 110 training data and 110 acquired.
	max y acquired =  45.71
	run 57
	diverse RF run
	eval budget 220 = 110 training data and 110 acquired.
	max y acquired =  45.71
	run 58
	diverse RF run
	eval budget 220 = 110 training data and 110 acquired.
	max y acquired =  45.71
	run 59
	diverse RF run
	eval budget 220 = 110 training data and 110 acquired.
	max y acquired =  45.71
	run 60
	diverse RF run
	eval budget 220 = 110 training data and 110 acquired.
	max y acquired =  45.71
	run 61
	diverse RF run
	eval budget 22

	max y acquired =  45.71
	run 31
	diverse RF run
	eval budget 240 = 120 training data and 120 acquired.
	max y acquired =  45.71
	run 32
	diverse RF run
	eval budget 240 = 120 training data and 120 acquired.
	max y acquired =  45.71
	run 33
	diverse RF run
	eval budget 240 = 120 training data and 120 acquired.
	max y acquired =  45.71
	run 34
	diverse RF run
	eval budget 240 = 120 training data and 120 acquired.
	max y acquired =  45.71
	run 35
	diverse RF run
	eval budget 240 = 120 training data and 120 acquired.
	max y acquired =  45.71
	run 36
	diverse RF run
	eval budget 240 = 120 training data and 120 acquired.
	max y acquired =  45.71
	run 37
	diverse RF run
	eval budget 240 = 120 training data and 120 acquired.
	max y acquired =  45.71
	run 38
	diverse RF run
	eval budget 240 = 120 training data and 120 acquired.
	max y acquired =  45.71
	run 39
	diverse RF run
	eval budget 240 = 120 training data and 120 acquired.
	max y acquired =  45.71
	run 40
	diverse RF run
	eval budget 24

	max y acquired =  45.71
	run 10
	diverse RF run
	eval budget 250 = 125 training data and 125 acquired.
	max y acquired =  45.71
	run 11
	diverse RF run
	eval budget 250 = 125 training data and 125 acquired.
	max y acquired =  45.71
	run 12
	diverse RF run
	eval budget 250 = 125 training data and 125 acquired.
	max y acquired =  45.71
	run 13
	diverse RF run
	eval budget 250 = 125 training data and 125 acquired.
	max y acquired =  45.71
	run 14
	diverse RF run
	eval budget 250 = 125 training data and 125 acquired.
	max y acquired =  45.71
	run 15
	diverse RF run
	eval budget 250 = 125 training data and 125 acquired.
	max y acquired =  45.71
	run 16
	diverse RF run
	eval budget 250 = 125 training data and 125 acquired.
	max y acquired =  45.71
	run 17
	diverse RF run
	eval budget 250 = 125 training data and 125 acquired.
	max y acquired =  45.71
	run 18
	diverse RF run
	eval budget 250 = 125 training data and 125 acquired.
	max y acquired =  45.71
	run 19
	diverse RF run
	eval budget 25

	max y acquired =  45.71
	run 89
	diverse RF run
	eval budget 250 = 125 training data and 125 acquired.
	max y acquired =  45.71
	run 90
	diverse RF run
	eval budget 250 = 125 training data and 125 acquired.
	max y acquired =  45.71
	run 91
	diverse RF run
	eval budget 250 = 125 training data and 125 acquired.
	max y acquired =  45.71
	run 92
	diverse RF run
	eval budget 250 = 125 training data and 125 acquired.
	max y acquired =  45.71
	run 93
	diverse RF run
	eval budget 250 = 125 training data and 125 acquired.
	max y acquired =  45.71
	run 94
	diverse RF run
	eval budget 250 = 125 training data and 125 acquired.
	max y acquired =  45.71
	run 95
	diverse RF run
	eval budget 250 = 125 training data and 125 acquired.
	max y acquired =  45.71
	run 96
	diverse RF run
	eval budget 250 = 125 training data and 125 acquired.
	max y acquired =  45.71
	run 97
	diverse RF run
	eval budget 250 = 125 training data and 125 acquired.
	max y acquired =  45.71
	run 98
	diverse RF run
	eval budget 25

save results to file

In [7]:
if not diversify_training:
    with open('rf_results.pkl', 'wb') as file:
        pickle.dump(rf_res, file)
else:
    with open('rf_div_results.pkl', 'wb') as file:
        pickle.dump(rf_res, file)